<a href="https://colab.research.google.com/github/W138923/GPT4-hackathon/blob/main/training_TD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
\import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque
from pandas import DataFrame as df
import pandas as pd
import seaborn as sns

# **0: envrioment**

import requests

url = 'https://www.alphavantage.co/query?function=FX_INTRADAY&from_symbol=JPY&to_symbol=USD&interval=60min&apikey=N914VF8FF269Z2ZJ&outputsize=full&datatype=csv'
r = requests.get(url)
import csv
data = []
for row in csv.reader(r.text.strip().splitlines()):
  data.append(row)
df(data[1:],columns=data[0])
map = df(data[1:],columns=data[0])
map = map.rename(columns={'open':'Open',
                    'high': 'High',
                    'low': 'Low',
                    'close': 'Close'})
map = map.drop('timestamp',axis=1)
for x in map.columns:
map.iloc[:,1:] = map.iloc[:,1:].astype('float64')

from alpha_vantage.foreignexchange import ForeignExchange
ts = ForeignExchange(key='N914VF8FF269Z2ZJ')
data, meta_data = ts.get_currency_exchange_intraday('USD','EUR',interval='1min', outputsize='full')
map = df(data).transpose()

In [ ]:
map = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DeepL/EN/TD3/USD_JPY Historical Data.csv')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
map = map.rename(columns={'open':'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}) 
map = map.drop('Date',axis=1) 
map = map.drop('Vol.',axis=1) 
map = map.drop('Change %',axis=1) 

for x in map.columns: 
  map.iloc[:,1:] = map.iloc[:,1:].astype('float32')

AttributeError: ignored

example = (5 - -5) * np.random.rand(10) + -5
_current_asset = 0
_account_balance = 1
test_df = map.iloc[:30,1].values
trade_asset =  t_asset * test_df[x]
for x in range(10):
  action = example[x]
  t_asset = action * _account_balance
  step_profit = 0 
  if _last_trade_price != 0: none
  
  _account_balance+=step_profit

In [ ]:
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
from enum import Enum
import matplotlib.pyplot as plt

class env_trade(gym.Env):
  metadata = {'render.modes': ['human']}
  # due to limitation of my programming skills, I will only allow one position to be existing at the time. Meaning, position will be reloved when another trade is done.
  # this only works, if the comoddity price is more then 0.

  def __init__(self, df, window_size=30,account_balance=1,max_leverage=5):
      # In order to add function to use mutiple df, df2 and df3 will be here, but unused.
    # what should be remembered?
    # parameter
    self._initial_balance=account_balance
    self._max_leverage = max_leverage 
    self.window_size = window_size
    self._sell_commission = 0.005
    self._leverage_commission = 0.001 # it will not be used for now.
    self._trade_commission = 0.005
    self._max_episode_steps= len(map)-window_size
    self.df = df
    self.signal_features=self.df.loc[:,'Price']
    # initial values
    self._account_balance= self._initial_balance
    self._invested = 0
    self._total_profit = 0 
    self._current_tick = self.window_size
    self._position_history = []
    self.history = None
    # spaces
    self.action_space = gym.spaces.Box(low=-(max_leverage), high=max_leverage, shape=(1,))
                                              #[0,1] )#0 is nothing, 1 is resolve previous trade
                                              #[0,self.map_number] #map number could be changed. and [0] is trade, also add code to change account_balance each time
    self.observation_space = gym.spaces.Box(low=self.df.min()['Low'], high=self.df.max()['High'], shape=[window_size,], dtype=np.float32) # maybe add observation about last trade
    
  
  def step(self, action):
    #prevent buying asset when account balance is negative.
    print("starting cash: %f" % (self._account_balance),"starting assets: %f" % (self._invested))
    step_profit = 0
    if self._account_balance <= 0 and action>=0:
      step_profit,t_asset = 0,0
    else:
      step_profit,self._invested,t_asset = self._trade(action)
      self._total_profit += step_profit
      self._account_balance += step_profit

    self._position_history.append([action,self._account_balance,t_asset]) # visualize later using the visual bubble. Second variable will be size of bubble.
    observation = self._get_observation()
    info = dict(
        total_profit = self._total_profit,
        position = action
    )
    print("step_profit: %f" % (step_profit))
    print("current cash: %f" % (self._account_balance),"current assets in cash worth: %f" % (self._invested/self.df.at[self._current_tick,"Price"]))
    print("ACTION: %f" % (action))
    print("---------------------")
    self._current_tick += 1
    done = False
    if self._current_tick==len(map):done = True
    if self._account_balance <= 0 and self._invested <=0: done =True
    if self._account_balance > 10e8: done = True
    if self._account_balance < -10e8: done = True
    if self._invested > 10e8: done = True
    return observation, step_profit, done, info
  
  def _trade(self,action):
    # should the profit be determined when the sales were made?
    c_invested = self._invested/self.df.at[self._current_tick,"Price"]
    if action <=0:      
      t_asset = self._account_balance*action *self.df.at[self._current_tick,"Price"]
      invested = self._resolution(t_asset)
    elif action 0>:
      t_asset = self._invested *action *self.df.at[self._current_tick,"Price"]
    #solve issue of selling amount being too dependent on current cash.
    profit =  self._account_balance*action*-1
    
    return profit,invested,t_asset

  def _resolution(self,t_asset):
    val1, val2 = False, False
    if t_asset >= 0: 
      val1 = True
    if self._invested >= 0: 
      val2 = True
    if val1 == val2 and val1 == True:
      new_delta = self._invested + t_asset
    elif val1 == val2 and val1 == False:
      new_delta = self._invested + t_asset
    elif val1 != val2:
      if t_asset >= self._invested:
        # check setting when value is negative. it is losing money for some reason.
        new_delta = abs(t_asset)-abs(self._invested)
        if val1 ==False: new_delta=new_delta * -1
      elif t_asset < self._invested:
       new_delta = abs(self._invested)-abs(t_asset)
       if val2 ==False: new_delta =new_delta  * -1
    else: print(val1,val2,t_asset,self._invested)
    return new_delta


  def _get_observation(self):
    observation = self.signal_features[(self._current_tick-self.window_size):self._current_tick]
    assert observation.shape[0] == 30
    return observation

  def reset(self):
    self._account_balance= self._initial_balance
    self._invested = 0
    self._total_profit = 0 
    self._current_tick = self.window_size
    self._position_history = []
    self.history = None

    observation = self._get_observation()

    return observation

account_balance = 1
env = env_trade(map,account_balance=account_balance,max_leverage=0.5)
avg_reward = 0.
eval_episodes = 1
memory=[]
for _ in range(eval_episodes):
  obs = env.reset()
  done = False
  while not done:
    action = env.action_space.sample()
    obs, reward, done, max_actions = env.step(action)
    memory.append([obs, reward, done, max_actions])
    avg_reward += reward
    if env._current_tick>40: done =True
avg_reward /= eval_episodes

# `1: Experience memory`

In [ ]:
class replay_memory(object):

  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

  def add(self, transition):
    if len(self.storage) == self.max_size:
      self.storage[int(self.ptr)] = transition
      self.ptr = (self.ptr + 1) % self.max_size
    else:
      self.storage.append(transition)

  def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storage), size=batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind: 
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False))
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)

# 2: Actor model & Actor

In [ ]:
class Actor(nn.Module):
  
  def __init__(self, state_dim, action_dim, max_action):
    super(Actor, self).__init__()
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.tanh(self.layer_3(x))
    return x

# 3: Two Critic Models & Critics

In [ ]:
class Critic(nn.Module):
  
  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    # Defining the first Critic neural network
    self.layer_1 = nn.Linear(state_dim + action_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, 1)
    # Defining the second Critic neural network
    self.layer_4 = nn.Linear(state_dim + action_dim, 400)
    self.layer_5 = nn.Linear(400, 300)
    self.layer_6 = nn.Linear(300, 1)

  def forward(self, x, u):
    xu = torch.cat([x, u], 1)
    # Forward-Propagation on the first Critic Neural Network
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    # Forward-Propagation on the second Critic Neural Network
    x2 = F.relu(self.layer_4(xu))
    x2 = F.relu(self.layer_5(x2))
    x2 = self.layer_6(x2)
    return x1, x2

  def Q1(self, x, u):
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    return x1

# 4: Training Process

In [ ]:
# Selecting the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Building the whole Training Process into a class
class TD3(object):
  def __init__(self, state_dim, action_dim, max_action):

    self.actor = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters())
    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
    self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state.reshape(1, -1)).to(device)
    print(self.actor(state).cpu().data.numpy().flatten())
    return self.actor(state).cpu().data.numpy().flatten()

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
    
    for it in range(iterations):
      
      # Step 4: We sample a batch of transitions (s, s’, a, r) from the memory
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      batch_rewards = np.array(batch_rewards, dtype=np.float32)
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)
      
      # Step 5: From the next state s’, the Actor target plays the next action a’
      next_action = self.actor_target(next_state)
      
      # Step 6: We add Gaussian noise to this next action a’ and we clamp it in a range of values supported by the environment
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)
      
      # Step 7: The two Critic targets take each the couple (s’, a’) as input and return two Q-values Qt1(s’,a’) and Qt2(s’,a’) as outputs
      target_Q1, target_Q2 = self.critic_target(next_state, next_action)
        #print(next_action.shape,next_state.shape)
      # Step 8: We keep the minimum of these two Q-values: min(Qt1, Qt2)
      target_Q = torch.min(target_Q1, target_Q2)
      
      # Step 9: We get the final target of the two Critic models, which is: Qt = r + γ * min(Qt1, Qt2), where γ is the discount factor
      target_Q = reward + ((1 - done) * discount * target_Q).detach()
      
      # Step 10: The two Critic models take each the couple (s, a) as input and return two Q-values Q1(s,a) and Q2(s,a) as outputs
      current_Q1, current_Q2 = self.critic(state, action)
      
      # Step 11: We compute the loss coming from the two Critic models: Critic Loss = MSE_Loss(Q1(s,a), Qt) + MSE_Loss(Q2(s,a), Qt)
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
      
      # Step 12: We backpropagate this Critic loss and update the parameters of the two Critic models with a SGD optimizer
      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()
      
      # Step 13: Once every two iterations, we update our Actor model by performing gradient ascent on the output of the first Critic model
      if it % policy_freq == 0:
        actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Step 14: Still once every two iterations, we update the weights of the Actor target by polyak averaging
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        
        # Step 15: Still once every two iterations, we update the weights of the Critic target by polyak averaging
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
  
  # Making a save method to save a trained model
  def save(self, filename, directory):
    torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
    torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))
  
  # Making a load method to load a pre-trained model
  def load(self, filename, directory):
    self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
    self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))

# 5: Evaluation

In [ ]:
def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(np.array(obs))
      obs, reward, done, max_actions = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward

In [ ]:
env_name = "trading" # Name of a environment (set it to any Continous environment you want)
seed = 0 # Random seed number
start_timesteps = 1e5 # Number of iterations/timesteps before which the model randomly chooses an action, and after which it starts to use the policy network
eval_freq = 5e3 # How often the evaluation step is performed (after how many timesteps)
max_timesteps = 5e5 # Total number of iterations/timesteps
save_models = True # Boolean checker whether or not to save the pre-trained model
expl_noise = 0.1 # Exploration noise - STD value of exploration Gaussian noise
batch_size = 100 # Size of the batch
discount = 0.99 # Discount factor gamma, used in the calculation of the total discounted reward
tau = 0.005 # Target network update rate
policy_noise = 0.2 # STD of Gaussian noise added to the actions for the exploration purposes
noise_clip = 0.5 # Maximum value of the Gaussian noise added to the actions (policy)
policy_freq = 2 # Number of iterations to wait before the policy network (Actor model) is updated

In [ ]:
if not os.path.exists("./results"):
  os.makedirs("./results")
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")

In [ ]:
account_balance = 1
max_lev = 0.5
env = env_trade(map,account_balance=account_balance,max_leverage=max_lev)

In [ ]:
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

In [ ]:
# policy net work (not actor critic)
policy = TD3(state_dim, action_dim, max_lev)
#reply memory creation
replay_buffer = replay_memory()
# list to store the score of the instance
evaluations = [evaluate_policy(policy)]
env.reset()

eval_episodes=1
avg_reward = 0.
for _ in range(eval_episodes):
  obs = env.reset()
  done = False
  while not done:
    action = policy.select_action(np.array(obs))
    obs, reward, done, max_actions = env.step(action)
    avg_reward += reward
avg_reward /= eval_episodes
print ("---------------------------------------")
print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
print ("---------------------------------------")

In [ ]:
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()
file_name = ''

In [ ]:
# We start the main loop over 500,000 timesteps

while total_timesteps < max_timesteps:
  
  # If the episode is done
  if done:

    # If we are not at the very beginning, we start the training process of the model
    if total_timesteps != 0:
      print("Total Timesteps: {} Episode Num: {} Reward: {}".format(total_timesteps, episode_num, episode_reward))
      policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)

    # We evaluate the episode and we save the policy
    if timesteps_since_eval >= eval_freq:
      timesteps_since_eval %= eval_freq
      evaluations.append(evaluate_policy(policy))
      policy.save(file_name, directory="./pytorch_models")
      np.save("./results/%s" % (file_name), evaluations)
    
    # When the training step is done, we reset the state of the environment
    obs = env.reset()
    
    # Set the Done to False
    done = False
    
    # Set rewards and episode timesteps to zero
    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1
  
  # Before 10000 timesteps, we play random actions
  if total_timesteps < start_timesteps:
    action = env.action_space.sample()
  else: # After 10000 timesteps, we switch to the model
    action = policy.select_action(np.array(obs))
    # If the explore_noise parameter is not 0, we add noise to the action and we clip it
    if expl_noise != 0:
      action = (action + np.random.normal(0, expl_noise, size=env.action_space.shape[0])).clip(env.action_space.low, env.action_space.high)
  
  # The agent performs the action in the environment, then reaches the next state and receives the reward
  new_obs, reward, done, max_action = env.step(action)
  
  # We check if the episode is done
  done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)
  
  # We increase the total reward
  episode_reward += reward
  
  # We store the new transition into the Experience Replay memory (ReplayBuffer)
  replay_buffer.add((obs, new_obs, action, reward, done_bool))

  # We update the state, the episode timestep, the total timesteps, and the timesteps since the evaluation of the policy
  obs = new_obs
  episode_timesteps += 1
  total_timesteps += 1
  timesteps_since_eval += 1

# We add the last policy evaluation to our list of evaluations and we save our model
evaluations.append(evaluate_policy(policy))
if save_models: policy.save("%s" % (file_name), directory="./pytorch_models")
np.save("./results/%s" % (file_name), evaluations)